In [1]:
import pandas as pd

df_idx = pd.read_csv('../data/processed/avg_ratings.csv')
df_corpus = pd.read_table('../data/raw/full_lyrics.tsv')

In [2]:
df_idx

,pair_id,sim_rating,song_1,song_2,length_1,length_2
0,10052_10842,1.333333,10052,10842,115,31
1,10052_11142,1.000000,10052,11142,115,105
2,10052_21856,1.000000,10052,21856,115,78
3,10052_24450,1.666667,10052,24450,115,108
4,10052_25047,0.000000,10052,25047,115,49
...,...,...,...,...,...,...
671,8863_55236,1.333333,8863,55236,64,61
672,9975_22822,2.333333,9975,22822,139,78
673,9975_24450,2.666667,9975,24450,139,108
674,9975_34125,0.000000,9975,34125,139,17


In [3]:
df_corpus

,song_id,text,artist_name,track_name
0,5439,"Dulce brillo\nque aún estaba afuera,\ncurtiend...",Eduardo Mateo,Dulce Brillo
1,29094,"No exactamente yo, aunque sí,\npero no lo soy....",Maga,"Sí, Pero No Lo Soy"
2,69790,"Mi novia Ximenita\nle gusta pelear,\npatea de ...",Los Piojos,Ximenita
3,66730,Oh de profundis.\nOh de procella.\nAh Canto.\n...,Arcana,Chant of the Awakening
4,28437,"Hasta la punta de tu luz,\nhasta tu grito más ...",Arbolito,La Máquina
...,...,...,...,...
70,38000,Me gusta cantarle al viento\nporque vuelan mis...,El Halcon De La Sierra,La Feria De Las Flores
71,48594,Y en la plaza de mi pueblo\ndijo el jornalero ...,Reincidentes,Jornaleros Andaluces
72,37194,Me voy a tomar\nese vino que pensás dejar\ny n...,Bulldog,Un Vinito Mas
73,25047,PI PA PU\nPA PA PI PI PA PIUI\nPI PA PI\nPA PI...,Los Visitantes,Pi-Pa-Pu


El script de _generate_examples en sts_dataset.py lee las columnas desde un tsv por líneas con separador '\t'. Asigna los elementos en la posición 0, 1 y 2 a sentence1, sentence2 y label respectivamente. Se han creado los archivos respetando esta estructura.

In [13]:
_df = df_corpus.set_index('song_id')
df = pd.DataFrame()
df['sentence1']=_df.loc[df_idx.song_1].text.reset_index(drop=['song_id'])
df['sentence2']=_df.loc[df_idx.song_2].text.reset_index(drop=['song_id'])
df.reset_index(inplace=True, drop=['index'])
df['label'] = df_idx.sim_rating.astype(float)


Se ha seguido el mismo split que en el paper de Maria y Agerri: 85% train, 5% dev, 10% test.

In [14]:
from sklearn.model_selection import train_test_split
train_dev, test = train_test_split(df, test_size=0.1, random_state=42, shuffle=True)
train, dev = train_test_split(df, test_size=0.0557, random_state=42, shuffle=True)

In [15]:
train

,sentence1,sentence2,label
672,Cuantas noches fatídicas de vicio\nTus ilusion...,Mirando la hiedra\ntu recuerdo me envenena.\nT...,2.333333
76,Sin un amor\nla vida no se llama vida.\nSin un...,"Adeste fideles\nLaeti triumphantes.\nVenite, v...",0.000000
584,Oh de profundis.\nOh de procella.\nAh Canto.\n...,Me gusta cantarle al viento\nporque vuelan mis...,0.000000
10,Bueno... Nunca me podré bajar\ndel planeta cer...,Me gusta cantarle al viento\nporque vuelan mis...,1.000000
507,"Dulce brillo\nque aún estaba afuera,\ncurtiend...",Si me pones atención\nTe puedo distraer.\nEscu...,1.333333
...,...,...,...
71,Sin un amor\nla vida no se llama vida.\nSin un...,"Busco cambiar de rumbo,\nbusco un poco de verd...",1.000000
106,La ví en la recepción\ncon un vaso de vino.\nS...,Moria... Moria... Moria... Moria...\nMoria... ...,0.000000
270,Quién me querrá\ncuando el cielo se vuelva osc...,Me voy a tomar\nese vino que pensás dejar\ny n...,1.000000
435,Suenan las campanas del alba\nen la plazuela.\...,Celebremos señores con gusto\neste día de plac...,3.333333


In [16]:
dev.shape

(38, 3)

In [17]:
test.shape

(68, 3)

In [20]:
train.to_csv('../sts/train.tsv', sep='\t', index=False)
dev.to_csv('../sts/valid.tsv', sep='\t', index=False)
test.to_csv('../sts/test.tsv', sep='\t', index=False)